<a href="https://colab.research.google.com/github/sabeesh90/Layer_Fine_Tuning_Deep_Neural_Networks/blob/main/The_Lazarus_Trials_Xception_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing libraries

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import seaborn as sns
import warnings
import os
from skimage import io
import pathlib
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix
!pip install -q -U keras-tuner
import kerastuner as kt
%load_ext tensorboard
warnings.filterwarnings("ignore")


     |████████████████████████████████| 96 kB 3.6 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
pip install Ipython

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
classes = ['GALAXY', 'QSO', 'STAR']

In [ ]:
def generate_graph(model_training):
    plt.figure(figsize= (15,5))
    plt.subplot(1,2,1)
    plt.plot(model_training.history.history['accuracy'])
    plt.plot(model_training.history.history['val_accuracy'])
    plt.title('Training vs TestAccuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(('Training', 'Validation'), loc = 'best')
    plt.subplot(1,2,2)
    plt.plot(model_training.history.history['loss'])
    plt.plot(model_training.history.history['val_loss'])
    plt.title('Training loss vs Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(('Training', 'Validation'), loc = 'best')
    plt.show()

class Metrics:
    # from sklearn
    def __init__(self, model, val_ds):
        self.val_ds = val_ds
        self.model = model
        self.prediction_classes = np.array([])
        self.true_classes = np.array([])
        
    def calculate_scores(self):     
        print('calculating score.....')   
        
        for x, y in self.val_ds:
            self.prediction_classes = np.concatenate([self.prediction_classes,np.argmax(self.model.predict(x), axis = -1)])
            self.true_classes = np.concatenate([self.true_classes, np.argmax(y.numpy(), axis=-1)])
        
        class_report = classification_report(self.true_classes, self.prediction_classes)
        conf_matrix = confusion_matrix(self.true_classes, self.prediction_classes)
        print(conf_matrix)
        print('------------------------------------')
        print(class_report)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

#### Xception

##### Data preparation

In [ ]:
# Preprocessing the data
dataset_path = 'Images'
img_width = 512
img_height = 512
channels = 3
num_classes = 3
# batch_size = 8
batch_size = 1 # for full model training

img_width_crop = 1024
img_height_crop = 1024

rescale = tf.keras.Sequential([layers.experimental.preprocessing.Rescaling(1./255)])


# augmenting the data
# cropless flip and rotate

data_aug_flip = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                # layers.experimental.preprocessing.RandomRotation(0.2)])
])

data_aug_flip_rotate = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                layers.experimental.preprocessing.RandomRotation(0.2)])


# crop flip and rotate
data_aug_crop_flip_rotate = tf.keras.Sequential([layers.experimental.preprocessing.CenterCrop(512,512),
                                                 layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                                 layers.experimental.preprocessing.RandomRotation(0.2)])

data_aug_crop_flip = tf.keras.Sequential([layers.experimental.preprocessing.CenterCrop(512,512),
                                          layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical")])

normal_crop = tf.keras.Sequential([layers.experimental.preprocessing.CenterCrop(512,512)],)
batch_size =32 
autotune = tf.data.AUTOTUNE

# Preparing the normal dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'training',seed = 123, image_size = (img_width, img_height), 
                                                                  batch_size = batch_size, label_mode = 'categorical',)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'validation',seed = 123, image_size = (img_width, img_height),
                                                                batch_size = batch_size, label_mode = 'categorical')

train_ds_cropped = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'training',seed = 123, image_size = (img_width_crop, img_height_crop), 
                                                                  batch_size = batch_size, label_mode = 'categorical',)
val_ds_cropped = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'validation',seed = 123, image_size = (img_width_crop, img_height_crop),
                                                                batch_size = batch_size, label_mode = 'categorical')


# preparing the scalred dataset
train_ds_scaled = train_ds.map(lambda x,y: (rescale(x, training = True), y), num_parallel_calls=autotune)
val_ds_scaled = val_ds.map(lambda x,y: (rescale(x, training = False), y), num_parallel_calls=autotune)
train_ds.prefetch(buffer_size=autotune)
val_ds.prefetch(buffer_size=autotune)

train_ds_scaled_cropped = train_ds_scaled.map(lambda x,y: (rescale(x, training = True), y), num_parallel_calls=autotune)
val_ds_scaled_cropped = val_ds_scaled.map(lambda x,y: (rescale(x, training = False), y), num_parallel_calls=autotune)

# augment function 
def prepare_dataset(ds, flip = False, flip_rotate = False):
    if flip == True:
        ds = ds.map(lambda x, y : (data_aug_flip(x, training = True), y), num_parallel_calls = autotune)
    elif flip_rotate == True:
        ds = ds.map(lambda x, y : (data_aug_flip_rotate(x, training = True), y), num_parallel_calls = autotune)
    
    return ds.prefetch(buffer_size =autotune)

def prepare_dataset_crop(ds, crop_flip = False, crop_flip_rotate = False):
    if crop_flip_rotate == True:
        ds = ds.map(lambda x, y : (data_aug_crop_flip_rotate(x, training = True), y), num_parallel_calls = autotune)
    elif crop_flip == True:
        ds = ds.map(lambda x, y : (data_aug_crop_flip(x, training = True), y), num_parallel_calls = autotune)
    
    return ds.prefetch(buffer_size =autotune)
    
def prepare_dataset_normal_crop(ds, training = True):
    ds = ds.map(lambda x, y : (normal_crop(x, training = training), y), num_parallel_calls = autotune)
    return ds.prefetch(buffer_size =autotune)
# 25 batches of 32 each and 7 batches of 32 each



Found 1000 files belonging to 3 classes.
Using 800 files for training.
Found 1000 files belonging to 3 classes.
Using 200 files for validation.
Found 1000 files belonging to 3 classes.
Using 800 files for training.
Found 1000 files belonging to 3 classes.
Using 200 files for validation.


##### Xception model building

In [ ]:
from tensorflow.keras.applications import  Xception
models = {'base_model'+str(num) : Xception(include_top=False, weights='imagenet', 
                                           input_shape = (img_height, img_width, 3),classes = num_classes, pooling = 'avg') for num in range(0,14)}

83697664/83683744 [==============================] - 0s 0us/step


In [ ]:
models

{'base_model0': <keras.engine.functional.Functional at 0x7f7efe10cc50>,
 'base_model1': <keras.engine.functional.Functional at 0x7f7eeca8fc90>,
 'base_model10': <keras.engine.functional.Functional at 0x7f7eab8dfcd0>,
 'base_model11': <keras.engine.functional.Functional at 0x7f7eab611850>,
 'base_model12': <keras.engine.functional.Functional at 0x7f7eab621b50>,
 'base_model13': <keras.engine.functional.Functional at 0x7f7eab2a8210>,
 'base_model2': <keras.engine.functional.Functional at 0x7f7eecaabd50>,
 'base_model3': <keras.engine.functional.Functional at 0x7f7eec9e8a50>,
 'base_model4': <keras.engine.functional.Functional at 0x7f7eec484410>,
 'base_model5': <keras.engine.functional.Functional at 0x7f7eecb8f190>,
 'base_model6': <keras.engine.functional.Functional at 0x7f7eec0623d0>,
 'base_model7': <keras.engine.functional.Functional at 0x7f7eabffec10>,
 'base_model8': <keras.engine.functional.Functional at 0x7f7eabc46c90>,
 'base_model9': <keras.engine.functional.Functional at 0x7f7

In [ ]:
layer_count  = len(models['base_model0'].layers)
print(f'There are a total of {layer_count} in the resnet layer')

There are a total of 133 in the resnet layer


In [ ]:
for model,value in models.items():
    print(model)

base_model0
base_model1
base_model2
base_model3
base_model4
base_model5
base_model6
base_model7
base_model8
base_model9
base_model10
base_model11
base_model12
base_model13


In [ ]:
num=0
for i, (model_name, model) in zip(range(133, -1,-2), models.items()):
    break_point  = i
    for layer in model.layers[0:break_point]:
        layer.trainable = False
    for layer in model.layers[break_point:]:
        layer.trainable = True
    input_layer = model.input
    output_layer = model.output
    x = Dense(num_classes, activation  = 'softmax')(output_layer)    
    models[model_name] = Model(inputs = input_layer, outputs = x)


In [ ]:
for model_name, model in models.items():
    count = 0
    trained_layers = ['a' if layer.trainable == True else None for layer in model.layers]
    print(f'The number of trainable layers in {model_name} is', trained_layers.count('a'), sep=':')

The number of trainable layers in base_model0 is:1
The number of trainable layers in base_model1 is:3
The number of trainable layers in base_model2 is:5
The number of trainable layers in base_model3 is:7
The number of trainable layers in base_model4 is:9
The number of trainable layers in base_model5 is:11
The number of trainable layers in base_model6 is:13
The number of trainable layers in base_model7 is:15
The number of trainable layers in base_model8 is:17
The number of trainable layers in base_model9 is:19
The number of trainable layers in base_model10 is:21
The number of trainable layers in base_model11 is:23
The number of trainable layers in base_model12 is:25
The number of trainable layers in base_model13 is:27


In [ ]:
# resnet_model.summary()
for model_name, model in models.items():
    model.compile(optimizer = optimizers.Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
i=0
for model_name, model in models.items():
    trainableParams = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    nonTrainableParams = np.sum([np.prod(v.get_shape()) for v in model.non_trainable_weights])
    totalParams = trainableParams + nonTrainableParams
    
    print('base_model'+str(i),trainableParams,nonTrainableParams,totalParams)
    i+=1


base_model0 6147 20861480 20867627
base_model1 6147 20861480 20867627
base_model2 3169795 17697832 20867627
base_model3 3172867 17694760 20867627
base_model4 4754947 16112680 20867627
base_model5 4756995 16110632 20867627
base_model6 5504515 15363112 20867627
base_model7 6256539 14611088 20867627
base_model8 6794531 14073096 20867627
base_model9 6794531 14073096 20867627
base_model10 7332523 13535104 20867627
base_model11 7333979 13533648 20867627
base_model12 7870515 12997112 20867627
base_model13 8408507 12459120 20867627


##### Xception model with normalization of  image pixels

In [ ]:
models['base_model0'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 152s 4s/step - loss: 1.6283 - accuracy: 0.5100 - val_loss: 1.0216 - val_accuracy: 0.4950
Epoch 2/60
25/25 [==============================] - 17s 570ms/step - loss: 1.5579 - accuracy: 0.6712 - val_loss: 1.0056 - val_accuracy: 0.8000
Epoch 3/60
25/25 [==============================] - 17s 570ms/step - loss: 1.5022 - accuracy: 0.8037 - val_loss: 0.9772 - val_accuracy: 0.8100
Epoch 4/60
25/25 [==============================] - 17s 569ms/step - loss: 1.4555 - accuracy: 0.8163 - val_loss: 0.9502 - val_accuracy: 0.7950
Epoch 5/60
25/25 [==============================] - 17s 570ms/step - loss: 1.4111 - accuracy: 0.8288 - val_loss: 0.9193 - val_accuracy: 0.8050
Epoch 6/60
25/25 [==============================] - 17s 568ms/step - loss: 1.3691 - accuracy: 0.8363 - val_loss: 0.8984 - val_accuracy: 0.8200
Epoch 7/60
25/25 [==============================] - 17s 571ms/step - loss: 1.3314 - accuracy: 0.8425 - val_loss: 0.8760 - val_accuracy: 0.8350
E

In [ ]:
models['base_model1'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 30, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr,early_stop])

Epoch 1/30
25/25 [==============================] - 19s 593ms/step - loss: 1.5521 - accuracy: 0.4925 - val_loss: 1.0451 - val_accuracy: 0.5500
Epoch 2/30
25/25 [==============================] - 17s 568ms/step - loss: 1.4867 - accuracy: 0.6888 - val_loss: 0.9956 - val_accuracy: 0.7000
Epoch 3/30
25/25 [==============================] - 17s 567ms/step - loss: 1.4391 - accuracy: 0.7538 - val_loss: 0.9563 - val_accuracy: 0.7300
Epoch 4/30
25/25 [==============================] - 17s 567ms/step - loss: 1.3960 - accuracy: 0.7625 - val_loss: 0.9207 - val_accuracy: 0.7450
Epoch 5/30
25/25 [==============================] - 17s 569ms/step - loss: 1.3544 - accuracy: 0.8062 - val_loss: 0.8912 - val_accuracy: 0.7700
Epoch 6/30
25/25 [==============================] - 17s 566ms/step - loss: 1.3172 - accuracy: 0.7812 - val_loss: 0.8693 - val_accuracy: 0.7850
Epoch 7/30
25/25 [==============================] - 17s 568ms/step - loss: 1.2810 - accuracy: 0.7862 - val_loss: 0.8467 - val_accuracy: 0.7900

In [ ]:
models['base_model2'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 21s 612ms/step - loss: 1.1912 - accuracy: 0.7138 - val_loss: 0.7440 - val_accuracy: 0.7850
Epoch 2/60
25/25 [==============================] - 17s 586ms/step - loss: 0.7656 - accuracy: 0.8238 - val_loss: 0.5737 - val_accuracy: 0.8150
Epoch 3/60
25/25 [==============================] - 17s 586ms/step - loss: 0.6139 - accuracy: 0.8775 - val_loss: 0.4907 - val_accuracy: 0.8300
Epoch 4/60
25/25 [==============================] - 17s 583ms/step - loss: 0.5411 - accuracy: 0.8900 - val_loss: 0.4350 - val_accuracy: 0.8400
Epoch 5/60
25/25 [==============================] - 17s 584ms/step - loss: 0.4874 - accuracy: 0.8925 - val_loss: 0.4178 - val_accuracy: 0.8400
Epoch 6/60
25/25 [==============================] - 17s 584ms/step - loss: 0.4274 - accuracy: 0.9187 - val_loss: 0.3899 - val_accuracy: 0.8500
Epoch 7/60
25/25 [==============================] - 17s 586ms/step - loss: 0.3878 - accuracy: 0.9275 - val_loss: 0.3761 - val_accuracy: 0.8700

In [ ]:
models['base_model3'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 614ms/step - loss: 1.2769 - accuracy: 0.5863 - val_loss: 0.7215 - val_accuracy: 0.7700
Epoch 2/60
25/25 [==============================] - 17s 589ms/step - loss: 0.8108 - accuracy: 0.8475 - val_loss: 0.5853 - val_accuracy: 0.8000
Epoch 3/60
25/25 [==============================] - 17s 589ms/step - loss: 0.6324 - accuracy: 0.8750 - val_loss: 0.4384 - val_accuracy: 0.8400
Epoch 4/60
25/25 [==============================] - 17s 588ms/step - loss: 0.5378 - accuracy: 0.8850 - val_loss: 0.3831 - val_accuracy: 0.8450
Epoch 5/60
25/25 [==============================] - 17s 589ms/step - loss: 0.4653 - accuracy: 0.9050 - val_loss: 0.3516 - val_accuracy: 0.8650
Epoch 6/60
25/25 [==============================] - 17s 590ms/step - loss: 0.3999 - accuracy: 0.9262 - val_loss: 0.3334 - val_accuracy: 0.8600
Epoch 7/60
25/25 [==============================] - 17s 588ms/step - loss: 0.3590 - accuracy: 0.9237 - val_loss: 0.3165 - val_accuracy: 0.8750

In [ ]:
models['base_model4'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 619ms/step - loss: 1.0660 - accuracy: 0.7912 - val_loss: 0.6664 - val_accuracy: 0.7500
Epoch 2/60
25/25 [==============================] - 18s 597ms/step - loss: 0.5596 - accuracy: 0.8612 - val_loss: 0.4320 - val_accuracy: 0.8500
Epoch 3/60
25/25 [==============================] - 18s 597ms/step - loss: 0.4147 - accuracy: 0.9200 - val_loss: 0.4827 - val_accuracy: 0.8200
Epoch 4/60
25/25 [==============================] - 18s 596ms/step - loss: 0.3053 - accuracy: 0.9325 - val_loss: 0.3852 - val_accuracy: 0.8450
Epoch 5/60
25/25 [==============================] - 17s 594ms/step - loss: 0.2579 - accuracy: 0.9513 - val_loss: 0.4201 - val_accuracy: 0.8300
Epoch 6/60
25/25 [==============================] - 17s 592ms/step - loss: 0.2263 - accuracy: 0.9488 - val_loss: 0.3249 - val_accuracy: 0.8700
Epoch 7/60
25/25 [==============================] - 18s 596ms/step - loss: 0.1800 - accuracy: 0.9625 - val_loss: 0.2885 - val_accuracy: 0.8800

In [ ]:
models['base_model5'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 21s 621ms/step - loss: 1.0875 - accuracy: 0.7738 - val_loss: 0.7122 - val_accuracy: 0.7450
Epoch 2/60
25/25 [==============================] - 18s 599ms/step - loss: 0.6225 - accuracy: 0.8662 - val_loss: 0.6602 - val_accuracy: 0.7200
Epoch 3/60
25/25 [==============================] - 18s 596ms/step - loss: 0.4419 - accuracy: 0.9225 - val_loss: 0.9909 - val_accuracy: 0.5650
Epoch 4/60
25/25 [==============================] - 18s 600ms/step - loss: 0.3619 - accuracy: 0.9312 - val_loss: 1.6731 - val_accuracy: 0.5550
Epoch 5/60
25/25 [==============================] - 18s 600ms/step - loss: 0.3038 - accuracy: 0.9413 - val_loss: 1.9383 - val_accuracy: 0.5450
Epoch 6/60
25/25 [==============================] - 18s 600ms/step - loss: 0.2433 - accuracy: 0.9513 - val_loss: 1.9858 - val_accuracy: 0.5350
Epoch 7/60
25/25 [==============================] - 17s 596ms/step - loss: 0.2068 - accuracy: 0.9550 - val_loss: 2.0865 - val_accuracy: 0.5350

In [ ]:
models['base_model6'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 626ms/step - loss: 0.9905 - accuracy: 0.7825 - val_loss: 0.6722 - val_accuracy: 0.7500
Epoch 2/60
25/25 [==============================] - 18s 598ms/step - loss: 0.5221 - accuracy: 0.9062 - val_loss: 0.5735 - val_accuracy: 0.8250
Epoch 3/60
25/25 [==============================] - 18s 600ms/step - loss: 0.3645 - accuracy: 0.9350 - val_loss: 0.4340 - val_accuracy: 0.8300
Epoch 4/60
25/25 [==============================] - 18s 599ms/step - loss: 0.2623 - accuracy: 0.9450 - val_loss: 0.4694 - val_accuracy: 0.8250
Epoch 5/60
25/25 [==============================] - 18s 600ms/step - loss: 0.2027 - accuracy: 0.9600 - val_loss: 0.5080 - val_accuracy: 0.8050
Epoch 6/60
25/25 [==============================] - 18s 606ms/step - loss: 0.1777 - accuracy: 0.9625 - val_loss: 0.7142 - val_accuracy: 0.7350
Epoch 7/60
25/25 [==============================] - 18s 600ms/step - loss: 0.1480 - accuracy: 0.9712 - val_loss: 0.6986 - val_accuracy: 0.7600

In [ ]:
models['base_model7'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 21s 642ms/step - loss: 1.0230 - accuracy: 0.7550 - val_loss: 0.6483 - val_accuracy: 0.7250
Epoch 2/60
25/25 [==============================] - 18s 614ms/step - loss: 0.4764 - accuracy: 0.8975 - val_loss: 0.4416 - val_accuracy: 0.8350
Epoch 3/60
25/25 [==============================] - 18s 614ms/step - loss: 0.3396 - accuracy: 0.9250 - val_loss: 0.5303 - val_accuracy: 0.7950
Epoch 4/60
25/25 [==============================] - 18s 615ms/step - loss: 0.2303 - accuracy: 0.9425 - val_loss: 0.8844 - val_accuracy: 0.6350
Epoch 5/60
25/25 [==============================] - 18s 618ms/step - loss: 0.1968 - accuracy: 0.9600 - val_loss: 1.3560 - val_accuracy: 0.5550
Epoch 6/60
25/25 [==============================] - 18s 611ms/step - loss: 0.1555 - accuracy: 0.9625 - val_loss: 1.2258 - val_accuracy: 0.5950
Epoch 7/60
25/25 [==============================] - 18s 616ms/step - loss: 0.1244 - accuracy: 0.9800 - val_loss: 0.9899 - val_accuracy: 0.6700

In [ ]:
models['base_model8'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 21s 661ms/step - loss: 1.0227 - accuracy: 0.7450 - val_loss: 0.4835 - val_accuracy: 0.8250
Epoch 2/60
25/25 [==============================] - 18s 628ms/step - loss: 0.4589 - accuracy: 0.8938 - val_loss: 0.3232 - val_accuracy: 0.8750
Epoch 3/60
25/25 [==============================] - 19s 637ms/step - loss: 0.3114 - accuracy: 0.9400 - val_loss: 0.2806 - val_accuracy: 0.8950
Epoch 4/60
25/25 [==============================] - 19s 635ms/step - loss: 0.2168 - accuracy: 0.9550 - val_loss: 0.2841 - val_accuracy: 0.8800
Epoch 5/60
25/25 [==============================] - 19s 641ms/step - loss: 0.1705 - accuracy: 0.9725 - val_loss: 0.3147 - val_accuracy: 0.8650
Epoch 6/60
25/25 [==============================] - 18s 633ms/step - loss: 0.1416 - accuracy: 0.9725 - val_loss: 0.2681 - val_accuracy: 0.9000
Epoch 7/60
25/25 [==============================] - 18s 632ms/step - loss: 0.1008 - accuracy: 0.9812 - val_loss: 0.2788 - val_accuracy: 0.8850

In [ ]:
models['base_model9'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 22s 658ms/step - loss: 1.0008 - accuracy: 0.7063 - val_loss: 0.6086 - val_accuracy: 0.7500
Epoch 2/60
25/25 [==============================] - 19s 635ms/step - loss: 0.4662 - accuracy: 0.9000 - val_loss: 0.3679 - val_accuracy: 0.8550
Epoch 3/60
25/25 [==============================] - 18s 635ms/step - loss: 0.2793 - accuracy: 0.9425 - val_loss: 0.4374 - val_accuracy: 0.8350
Epoch 4/60
25/25 [==============================] - 19s 634ms/step - loss: 0.2192 - accuracy: 0.9475 - val_loss: 0.5272 - val_accuracy: 0.8050
Epoch 5/60
25/25 [==============================] - 18s 634ms/step - loss: 0.1691 - accuracy: 0.9750 - val_loss: 0.5020 - val_accuracy: 0.8150
Epoch 6/60
25/25 [==============================] - 18s 634ms/step - loss: 0.1058 - accuracy: 0.9837 - val_loss: 0.5206 - val_accuracy: 0.8100
Epoch 7/60
25/25 [==============================] - 18s 634ms/step - loss: 0.0875 - accuracy: 0.9837 - val_loss: 0.5972 - val_accuracy: 0.7850

In [ ]:
models['base_model10'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 22s 681ms/step - loss: 1.0019 - accuracy: 0.7375 - val_loss: 0.5373 - val_accuracy: 0.8200
Epoch 2/60
25/25 [==============================] - 19s 657ms/step - loss: 0.4712 - accuracy: 0.9075 - val_loss: 0.3388 - val_accuracy: 0.8650
Epoch 3/60
25/25 [==============================] - 19s 655ms/step - loss: 0.2953 - accuracy: 0.9425 - val_loss: 0.3355 - val_accuracy: 0.8550
Epoch 4/60
25/25 [==============================] - 19s 653ms/step - loss: 0.2083 - accuracy: 0.9638 - val_loss: 0.4178 - val_accuracy: 0.8400
Epoch 5/60
25/25 [==============================] - 19s 651ms/step - loss: 0.1246 - accuracy: 0.9787 - val_loss: 0.3941 - val_accuracy: 0.8450
Epoch 6/60
25/25 [==============================] - 19s 653ms/step - loss: 0.0968 - accuracy: 0.9837 - val_loss: 0.4563 - val_accuracy: 0.8500
Epoch 7/60
25/25 [==============================] - 19s 653ms/step - loss: 0.0702 - accuracy: 0.9900 - val_loss: 0.6294 - val_accuracy: 0.7650

In [ ]:
models['base_model11'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 22s 682ms/step - loss: 0.9628 - accuracy: 0.8100 - val_loss: 0.5963 - val_accuracy: 0.7700
Epoch 2/60
25/25 [==============================] - 20s 693ms/step - loss: 0.4234 - accuracy: 0.9112 - val_loss: 0.3365 - val_accuracy: 0.8700
Epoch 3/60
25/25 [==============================] - 19s 662ms/step - loss: 0.2653 - accuracy: 0.9513 - val_loss: 0.3007 - val_accuracy: 0.8850
Epoch 4/60
25/25 [==============================] - 19s 657ms/step - loss: 0.1766 - accuracy: 0.9638 - val_loss: 0.3158 - val_accuracy: 0.8650
Epoch 5/60
25/25 [==============================] - 19s 657ms/step - loss: 0.1275 - accuracy: 0.9800 - val_loss: 0.4524 - val_accuracy: 0.8300
Epoch 6/60
25/25 [==============================] - 19s 659ms/step - loss: 0.0885 - accuracy: 0.9875 - val_loss: 0.3941 - val_accuracy: 0.8450
Epoch 7/60
25/25 [==============================] - 19s 656ms/step - loss: 0.0627 - accuracy: 0.9925 - val_loss: 0.4750 - val_accuracy: 0.8200

In [ ]:
models['base_model12'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 22s 695ms/step - loss: 0.9959 - accuracy: 0.7862 - val_loss: 0.5528 - val_accuracy: 0.7950
Epoch 2/60
25/25 [==============================] - 19s 667ms/step - loss: 0.4248 - accuracy: 0.9200 - val_loss: 0.3117 - val_accuracy: 0.8900
Epoch 3/60
25/25 [==============================] - 20s 673ms/step - loss: 0.2392 - accuracy: 0.9575 - val_loss: 0.2941 - val_accuracy: 0.8850
Epoch 4/60
25/25 [==============================] - 19s 671ms/step - loss: 0.1639 - accuracy: 0.9700 - val_loss: 0.3868 - val_accuracy: 0.8450
Epoch 5/60
25/25 [==============================] - 19s 670ms/step - loss: 0.1188 - accuracy: 0.9737 - val_loss: 0.3559 - val_accuracy: 0.8750
Epoch 6/60
25/25 [==============================] - 19s 669ms/step - loss: 0.0776 - accuracy: 0.9862 - val_loss: 0.4860 - val_accuracy: 0.8300
Epoch 7/60
25/25 [==============================] - 19s 672ms/step - loss: 0.0748 - accuracy: 0.9900 - val_loss: 0.6916 - val_accuracy: 0.7450

In [ ]:
models['base_model13'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 23s 712ms/step - loss: 1.0017 - accuracy: 0.7738 - val_loss: 0.7257 - val_accuracy: 0.6550
Epoch 2/60
25/25 [==============================] - 20s 684ms/step - loss: 0.4090 - accuracy: 0.9200 - val_loss: 0.3981 - val_accuracy: 0.8650
Epoch 3/60
25/25 [==============================] - 20s 686ms/step - loss: 0.2195 - accuracy: 0.9575 - val_loss: 0.3893 - val_accuracy: 0.8500
Epoch 4/60
25/25 [==============================] - 20s 689ms/step - loss: 0.1276 - accuracy: 0.9762 - val_loss: 0.5603 - val_accuracy: 0.7950
Epoch 5/60
25/25 [==============================] - 20s 687ms/step - loss: 0.0796 - accuracy: 0.9837 - val_loss: 0.6651 - val_accuracy: 0.7550
Epoch 6/60
25/25 [==============================] - 20s 687ms/step - loss: 0.0322 - accuracy: 0.9950 - val_loss: 1.1527 - val_accuracy: 0.6400
Epoch 7/60
25/25 [==============================] - 20s 686ms/step - loss: 0.0189 - accuracy: 0.9987 - val_loss: 0.9679 - val_accuracy: 0.7100

In [ ]:
from tensorflow.keras.applications import  Xception
base_xception= Xception(include_top=False, weights='imagenet',input_shape = (img_height, img_width, 3),classes = num_classes, pooling = 'avg')
for layer in base_xception.layers:
    layer.trainable = True
input_layer = base_xception.input
output_layer = base_xception.output
x = Dense(num_classes, activation  = 'softmax')(output_layer)    
base_xception_retrained = Model(inputs = input_layer, outputs = x)

trained_layers = ['a' if layer.trainable == True else None for layer in base_xception_retrained.layers]
print(f'The number of trainable layers in base_xception_retrained is', trained_layers.count('a'), sep=':')

The number of trainable layers in base_xception_retrained is:134


In [ ]:
base_xception_retrained.compile(optimizer = optimizers.Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
base_xception_retrained.fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60


ResourceExhaustedError: ignored